<a href="https://colab.research.google.com/github/sprabhala-cpu/Machine-Learning/blob/main/data_cleaning_with_pipleline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'

--2021-05-12 09:10:58--  https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30286 (30K) [application/x-httpd-php]
Saving to: ‘auto-mpg.data’

auto-mpg.data       100%[===================>]  29.58K  --.-KB/s    in 0.1s    

2021-05-12 09:10:59 (226 KB/s) - ‘auto-mpg.data’ saved [30286/30286]



In [12]:
import pandas as pd
cols = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
data=pd.read_csv('/content/auto-mpg.data',sep=' ',names=cols,comment='\t',skipinitialspace=True,na_values='?')

In [13]:
data

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1
...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1


In [14]:
#viewing data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MPG           398 non-null    float64
 1   Cylinders     398 non-null    int64  
 2   Displacement  398 non-null    float64
 3   Horsepower    392 non-null    float64
 4   Weight        398 non-null    float64
 5   Acceleration  398 non-null    float64
 6   Model Year    398 non-null    int64  
 7   Origin        398 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 25.0 KB


In [16]:
from sklearn.model_selection import StratifiedShuffleSplit
split=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in split.split(data,data['Cylinders']):
  strat_train_set=data.loc[train_index]
  strat_test_set=data.loc[test_index]

In [19]:
#segreagte the features and target variable
data=strat_train_set.drop('MPG',axis=1)
data_label=strat_train_set['MPG'].copy()

In [22]:
data.sample(5)

,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
93,8,318.0,150.0,4237.0,14.5,73,1
205,4,97.0,75.0,2155.0,16.4,76,3
61,4,122.0,86.0,2226.0,16.5,72,1
176,6,232.0,90.0,3211.0,17.0,75,1
166,8,302.0,129.0,3169.0,12.0,75,1


In [23]:
#the origin is country code we have to add country code
def preprocess_origin_cols(df):
  df['Origin']=df['Origin'].map({1:'India',2:'USA',3:'Germany'})
  return df

In [37]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
#create new columns using baseestimator and tranformermixin


In [29]:
import numpy as np
acc_ix,hpower_ix,cyl_ix=4,2,0
class CustomAttrAdder(BaseEstimator,TransformerMixin):
  def __init__(self,acc_on_power=True):
    self.acc_on_power=acc_on_power
  def fit(self,X,y=None):
    return self
  def transform(self,X):
    acc_on_cyl = X[:, acc_ix] / X[:, cyl_ix]
    if self.acc_on_power:
      acc_on_power=X[:,acc_ix]/X[:,hpower_ix]
      return np.c_[X,acc_on_power,acc_on_cyl]
    return np.c_[X,acc_on_cyl] 

In [43]:
from sklearn.pipeline import Pipeline
def num_pipeline_transformer(data):
  '''"function to process numerical tranformations"
      argument:
      data:original data
      returens 
      num_attrs:numerical dataframe
      num_pipeline:numerical pipelineobjec'''
  numerics=["float64",'int64']
  num_attrs=data.select_dtypes(include=numerics)
  num_pipeline=Pipeline([
          ('imputer',SimpleImputer(strategy='median')),
          ('attrs_adder',CustomAttrAdder()),
          ('std_scaler',StandardScaler())
  ])
  return num_attrs,num_pipeline

  

In [46]:
def pipeline_transformer(data):
  '''
  complete transforamtion for pipeline for both numnerical and categoricaldata
  argumer:
  data:original data
  reutrns 
  prepared data: ready to use'''
  cat_attrs=['Origin']
  num_attrs,num_pipeline=num_pipeline_transformer(data)
  full_pipeline=ColumnTransformer([
    ('num',num_pipeline,list(num_attrs)),
    ('cat',OneHotEncoder(),cat_attrs),                          
  ])
  prepared_data=full_pipeline.fit_transform(data)
  return prepared_data

In [47]:
preprocessed_df=preprocess_origin_cols(data)

In [48]:
preprocessed_df

,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
145,4,83.0,61.0,2003.0,19.0,74,Germany
151,4,79.0,67.0,2000.0,16.0,74,USA
388,4,156.0,92.0,2585.0,14.5,82,India
48,6,250.0,88.0,3139.0,14.5,71,India
114,4,98.0,90.0,2265.0,15.5,73,USA
...,...,...,...,...,...,...,...
147,4,90.0,75.0,2108.0,15.5,74,USA
156,8,400.0,170.0,4668.0,11.5,75,India
395,4,135.0,84.0,2295.0,11.6,82,India
14,4,113.0,95.0,2372.0,15.0,70,Germany


In [49]:
prepared=pipeline_transformer(preprocessed_df)

In [51]:
prepared[0]

array([-0.85657842, -1.07804475, -1.15192977, -1.17220298,  1.21586943,
       -0.54436373,  1.70952741,  1.29565517,  1.        ,  0.        ,
        0.        ])

In [52]:
#applying linear regression
from sklearn.linear_model import LinearRegression
lin_reg=LinearRegression()
lin_reg.fit(prepared,data_label)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [53]:
#testinf predictions with 
sample_data=data.iloc[:5]
sample_label=data_label.iloc[:5]


In [55]:
sample_data_prepared=pipeline_transformer(sample_data)

In [56]:
print('predicted are ',lin_reg.predict(sample_data_prepared))

predicted are  [29.08069379 27.78336755 26.08031176 12.70419279 22.23454159]


In [58]:
print('original are ',list (sample_label))

original are  [32.0, 31.0, 26.0, 18.0, 26.0]


In [61]:
#evaluation of model
from sklearn.metrics import mean_squared_error
mpg_predictions=lin_reg.predict(prepared)
lin_mse=mean_squared_error(data_label,mpg_predictions)
print(lin_mse)

8.755919038823139


In [62]:
mpg_prediction_sample=lin_reg.predict(sample_data_prepared)
lin_mse1=mean_squared_error(sample_label,mpg_prediction_sample)
print(lin_mse1)

12.219954815976802


In [65]:
#we calculate root mean squared error
lin_rmse=np.round(np.sqrt(lin_mse),4)

print(lin_rmse)
#rmse for linear regression is 2.95

2.959


In [68]:
from sklearn.tree import DecisionTreeRegressor
tree_reg=DecisionTreeRegressor()
tree_reg.fit(prepared, data_label)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [69]:
mpg_predictions = tree_reg.predict(prepared)
tree_mse = mean_squared_error(data_label, mpg_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [ ]:
#cross validation as we are getting rmse as 0.0 which means it is overfitted
#scikit learn k-fold cross validation splits the training set into k distinct su
#subsets called fols then it trains and evaluate the model k times  picking differe
'''fold for evaluation and traing on teh k-1 folds'''

In [71]:
from sklearn.model_selection import cross_val_score
scores=cross_val_score(tree_reg,prepared,data_label,scoring='neg_mean_squared_error',
                       cv=10)
tree_reg_rmse_scores=np.sqrt(-scores)

In [74]:
tree_reg_rmse_scores.mean()

3.2649258282472546

In [73]:
scores

array([ -9.91125   ,  -8.524375  ,  -7.7375    , -11.195     ,
        -5.2025    ,  -9.23      , -13.203125  , -20.55875   ,
       -18.85580645,  -6.88935484])

In [75]:
from sklearn.ensemble import RandomForestRegressor
forest_reg=RandomForestRegressor()
forest_reg_scores=cross_val_score(forest_reg,prepared,data_label,
                                  scoring='neg_mean_squared_error',
                                  cv=10)
forest_reg_rmse_scores=np.sqrt(-forest_reg_scores)

In [76]:
forest_reg_rmse_scores.mean()

2.6085637601165397

In [77]:
#after applying linear regrssion, decison tree and random forest 
'''we can see that random forest performed better but it can be fine tuned'''


'we can see that random forest performed better but it can be fine tuned'

In [78]:
from sklearn.model_selection import GridSearchCV

In [79]:
#we will use grid search to find best combination of hyperparameters for 
#random forest:

param_grid=[
            {'n_estimators':[3,5,10,30,100],'max_features':[2,4,6,8]},
            {'bootstrap':[False],'n_estimators':[3,10]}
]
forest_Reg=RandomForestRegressor()
grid_search=GridSearchCV(forest_Reg,param_grid,scoring='neg_mean_squared_error',cv=10)

In [80]:
grid_search.fit(prepared,data_label)

GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_j

In [81]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 100}

In [82]:
grid_search.best_score_

-7.039771515927418

In [83]:
cv_scores=grid_search.cv_results_

In [86]:
cv_scores['params']

[{'max_features': 2, 'n_estimators': 3},
 {'max_features': 2, 'n_estimators': 5},
 {'max_features': 2, 'n_estimators': 10},
 {'max_features': 2, 'n_estimators': 30},
 {'max_features': 2, 'n_estimators': 100},
 {'max_features': 4, 'n_estimators': 3},
 {'max_features': 4, 'n_estimators': 5},
 {'max_features': 4, 'n_estimators': 10},
 {'max_features': 4, 'n_estimators': 30},
 {'max_features': 4, 'n_estimators': 100},
 {'max_features': 6, 'n_estimators': 3},
 {'max_features': 6, 'n_estimators': 5},
 {'max_features': 6, 'n_estimators': 10},
 {'max_features': 6, 'n_estimators': 30},
 {'max_features': 6, 'n_estimators': 100},
 {'max_features': 8, 'n_estimators': 3},
 {'max_features': 8, 'n_estimators': 5},
 {'max_features': 8, 'n_estimators': 10},
 {'max_features': 8, 'n_estimators': 30},
 {'max_features': 8, 'n_estimators': 100},
 {'bootstrap': False, 'n_estimators': 3},
 {'bootstrap': False, 'n_estimators': 10}]

In [96]:

for mean_score,params in zip(cv_scores['mean_test_score'], cv_scores["params"]):
  print((np.sqrt(-mean_score)),params)

3.4658743084858434 {'max_features': 2, 'n_estimators': 3}
3.2976136594170935 {'max_features': 2, 'n_estimators': 5}
2.92627667203399 {'max_features': 2, 'n_estimators': 10}
2.9857970094619675 {'max_features': 2, 'n_estimators': 30}
2.8204059087416766 {'max_features': 2, 'n_estimators': 100}
3.6453002682493696 {'max_features': 4, 'n_estimators': 3}
3.133743677374375 {'max_features': 4, 'n_estimators': 5}
2.9085761329158535 {'max_features': 4, 'n_estimators': 10}
2.791000975236734 {'max_features': 4, 'n_estimators': 30}
2.693700082615109 {'max_features': 4, 'n_estimators': 100}
2.9305123172845176 {'max_features': 6, 'n_estimators': 3}
3.0583772908563467 {'max_features': 6, 'n_estimators': 5}
2.8398658000153416 {'max_features': 6, 'n_estimators': 10}
2.6626225792958316 {'max_features': 6, 'n_estimators': 30}
2.6722764423190553 {'max_features': 6, 'n_estimators': 100}
3.0596597904267266 {'max_features': 8, 'n_estimators': 3}
2.9941186031414015 {'max_features': 8, 'n_estimators': 5}
2.80420